In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/Voice_coversion

/content/gdrive/MyDrive/Voice_coversion


In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla P100-PCIE-16GB


In [4]:
import os
import pickle
from speaker_encoder import D_VECTOR
from collections import OrderedDict
import numpy as np
import torch

In [5]:
#check the data
rootDir = './spmel_16khz'
dirName, subdirList, _ = next(os.walk(rootDir))
print('Found directory: %s' % dirName)

drop_point = 30
subdirList = sorted(subdirList)[:drop_point]

print(subdirList)
print(len(subdirList))

Found directory: ./spmel_16khz
['p225', 'p226', 'p227', 'p228', 'p229', 'p230', 'p231', 'p232', 'p233', 'p234', 'p236', 'p237', 'p238', 'p239', 'p240', 'p241', 'p243', 'p244', 'p245', 'p246', 'p247', 'p248', 'p249', 'p250', 'p251', 'p252', 'p253', 'p254', 'p255', 'p256']
30


In [7]:
C = D_VECTOR(dim_input=80, dim_cell=768, dim_emb=256).eval().cuda()
c_checkpoint = torch.load('checkpoints/Speaker/3000000-BL.ckpt')
new_state_dict = OrderedDict()
for key, val in c_checkpoint['model_b'].items():
    new_key = key[7:]
    new_state_dict[new_key] = val
C.load_state_dict(new_state_dict)
num_uttrs = 10
len_crop = 128

speakers = []
print("Number of speakers: {}".format(len(subdirList)))
for speaker in sorted(subdirList):
    utterances = []
    utterances.append(speaker)
    _, _, fileList = next(os.walk(os.path.join(dirName,speaker)))
    print('Processing speaker:{}, num utterances: {}'.format(speaker, len(fileList)))
    # make speaker embedding
    assert len(fileList) >= num_uttrs
    idx_uttrs = np.random.choice(len(fileList), size=num_uttrs, replace=False)
    embs = []
    for i in range(num_uttrs):
        tmp = np.load(os.path.join(dirName, speaker, fileList[idx_uttrs[i]]))
        candidates = np.delete(np.arange(len(fileList)), idx_uttrs)
        # choose another utterance if the current one is too short
        while tmp.shape[0] < len_crop:
            idx_alt = np.random.choice(candidates)
            tmp = np.load(os.path.join(dirName, speaker, fileList[idx_alt]))
            candidates = np.delete(candidates, np.argwhere(candidates==idx_alt))
        left = np.random.randint(0, tmp.shape[0]-len_crop)
        melsp = torch.from_numpy(tmp[np.newaxis, left:left+len_crop, :]).cuda()
        emb = C(melsp)
        embs.append(emb.detach().squeeze().cpu().numpy())     
    utterances.append(np.mean(embs, axis=0))
    
    # create file list
    for fileName in sorted(fileList):
        utterances.append(os.path.join(speaker,fileName))
    speakers.append(utterances)
    
with open(os.path.join(rootDir, 'train.pkl'), 'wb') as handle:
    pickle.dump(speakers, handle)

Number of speakers: 30
Processing speaker:p225, num utterances: 462
Processing speaker:p226, num utterances: 712
Processing speaker:p227, num utterances: 778
Processing speaker:p228, num utterances: 732
Processing speaker:p229, num utterances: 758
Processing speaker:p230, num utterances: 794
Processing speaker:p231, num utterances: 912
Processing speaker:p232, num utterances: 822
Processing speaker:p233, num utterances: 744
Processing speaker:p234, num utterances: 714
Processing speaker:p236, num utterances: 984
Processing speaker:p237, num utterances: 678
Processing speaker:p238, num utterances: 908
Processing speaker:p239, num utterances: 1004
Processing speaker:p240, num utterances: 754
Processing speaker:p241, num utterances: 706
Processing speaker:p243, num utterances: 786
Processing speaker:p244, num utterances: 840
Processing speaker:p245, num utterances: 708
Processing speaker:p246, num utterances: 716
Processing speaker:p247, num utterances: 950
Processing speaker:p248, num ut